## Collecting data from Reddit

At first,I thought I would use the Python Reddit API Wrapper, but the limits imposed by Reddit on crawling are not the most friendly. To collect bulk amounts of data. Instead, I found a data dump of Reddit Comments separated by month as [JSON files](https://files.pushshift.io/reddit/comments/) , then I had to extract comment-reply pairs from the JSON file. for some comments, i found only 1 reply per comment. Even though many single comments might have many replies, so i decided to go with the top-voted one.

In [49]:
# import libaraies
import sqlite3
import json
from datetime import datetime

In [50]:
# gloabel variable
start_row = 0
cleanup = 1000000
timeframe = '2018-06'
sql_transaction = []

In [51]:
# intilaize connection , database for each month
# because it is huge amount of data (more than 20,000,000 recored)
connection = sqlite3.connect('data/{}.db'.format(timeframe))
c = connection.cursor()

In [52]:
# create table for each month
def create_table():
    c.execute("CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)")

In [53]:
# clean data
def clean_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [54]:
# return text of given comment id
def get_text(commentid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(commentid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: 
            return False
    except Exception as e:
        return False

In [55]:
# return attached score for parent if exist
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: 
            return False
    except Exception as e:
        return False

In [56]:
# check comment length and if the text is deleted
def acceptable(data):
    if len(data.split(' ')) > 1000 or len(data) < 1:
        return False
    elif len(data) > 32000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [57]:
# Start database connection the excute transactions
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [58]:
# insert new comment or update in the database
def sql_insert_comment(trans_type, commentid, parentid, parent, comment, subreddit, time, score):
    '''
    *args:
        trans_type:
                    1: update
                    2: insert
    '''
    try:
        if trans_type == 1: # update
            sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        else: # insert
            sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score) 
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [59]:
if __name__ == '__main__':
    create_table()
    # normal row counter
    row_counter = 0
    # counter for comments with replaies (complete pair)
    paired_rows = 0


    with open('data/RC_{}'.format(timeframe), buffering=1000) as f:
        for row in f:
            row_counter += 1

            if row_counter > start_row:
                try:
                    # read dict
                    row = json.loads(row)
                    # fetch needed info
                    parent_id = row['parent_id'].split('_')[1]
                    body = clean_data(row['body'])
                    created_utc = row['created_utc']
                    score = row['score']
                    comment_id = row['id']
                    subreddit = row['subreddit']
                    
                    # trying to find data of this comment's parent
                    parent_data = get_text(parent_id)
                    
                    # insert only valid body length
                    if acceptable(body):
                        # check if already exist a pair (parent,replay)
                        existing_comment_score = find_existing_score(parent_id)
                        
                        if existing_comment_score:
                            # current comment is better
                            if score > existing_comment_score:
                                # update this row
                                sql_insert_comment(1, comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                        # new comment (row)
                        elif parent_data:
                            if score >= 2:
                                # inset new row
                                sql_insert_comment(2, comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
                                paired_rows += 1
                        else:
                            sql_insert_comment(2, comment_id,parent_id,None,body,subreddit,created_utc,score)
                except Exception as e:
                    print(str(e))
                            
            if row_counter % 100000 == 0:
                print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))
                #break # should remove it to parse whole file
                
            if row_counter > start_row:
                if row_counter % cleanup == 0:
                    print("Cleanin up!")
                    sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                    c.execute(sql)
                    connection.commit()
                    c.execute("VACUUM")
                    connection.commit()

Total Rows Read: 100000, Paired Rows: 5940, Time: 2018-10-03 00:37:53.171778
Total Rows Read: 200000, Paired Rows: 14262, Time: 2018-10-03 00:38:07.726624
Total Rows Read: 300000, Paired Rows: 22539, Time: 2018-10-03 00:38:21.945287
Total Rows Read: 400000, Paired Rows: 30792, Time: 2018-10-03 00:38:35.866404
Total Rows Read: 500000, Paired Rows: 39295, Time: 2018-10-03 00:38:49.621815
Total Rows Read: 600000, Paired Rows: 46713, Time: 2018-10-03 00:39:02.687643
Total Rows Read: 700000, Paired Rows: 54537, Time: 2018-10-03 00:39:16.112981
Total Rows Read: 800000, Paired Rows: 63647, Time: 2018-10-03 00:39:30.244702
Total Rows Read: 900000, Paired Rows: 73143, Time: 2018-10-03 00:39:45.036768
Total Rows Read: 1000000, Paired Rows: 82662, Time: 2018-10-03 00:39:59.558898
Cleanin up!
Total Rows Read: 1100000, Paired Rows: 92146, Time: 2018-10-03 00:40:16.129319
Total Rows Read: 1200000, Paired Rows: 101599, Time: 2018-10-03 00:40:29.409104
Total Rows Read: 1300000, Paired Rows: 111580, Ti

Total Rows Read: 10300000, Paired Rows: 1054909, Time: 2018-10-03 01:03:57.757080
Total Rows Read: 10400000, Paired Rows: 1064804, Time: 2018-10-03 01:04:11.152173
Total Rows Read: 10500000, Paired Rows: 1074858, Time: 2018-10-03 01:04:24.481328
Total Rows Read: 10600000, Paired Rows: 1085027, Time: 2018-10-03 01:04:37.479376
Total Rows Read: 10700000, Paired Rows: 1095722, Time: 2018-10-03 01:04:50.138318
Total Rows Read: 10800000, Paired Rows: 1106304, Time: 2018-10-03 01:05:03.238303
Total Rows Read: 10900000, Paired Rows: 1117214, Time: 2018-10-03 01:05:16.283651
Total Rows Read: 11000000, Paired Rows: 1128238, Time: 2018-10-03 01:05:29.376822
Cleanin up!
Total Rows Read: 11100000, Paired Rows: 1139351, Time: 2018-10-03 01:06:07.014208
Total Rows Read: 11200000, Paired Rows: 1150385, Time: 2018-10-03 01:06:20.195586
Total Rows Read: 11300000, Paired Rows: 1161539, Time: 2018-10-03 01:06:33.271625
Total Rows Read: 11400000, Paired Rows: 1172796, Time: 2018-10-03 01:06:46.519573
Tota

Total Rows Read: 20200000, Paired Rows: 2113072, Time: 2018-10-03 01:34:08.511248
Total Rows Read: 20300000, Paired Rows: 2121933, Time: 2018-10-03 01:34:20.213323
Total Rows Read: 20400000, Paired Rows: 2131963, Time: 2018-10-03 01:34:32.747373
Total Rows Read: 20500000, Paired Rows: 2142240, Time: 2018-10-03 01:34:45.409359
Total Rows Read: 20600000, Paired Rows: 2152278, Time: 2018-10-03 01:34:58.157202
Total Rows Read: 20700000, Paired Rows: 2162136, Time: 2018-10-03 01:35:10.900698
Total Rows Read: 20800000, Paired Rows: 2172111, Time: 2018-10-03 01:35:23.676651
Total Rows Read: 20900000, Paired Rows: 2181844, Time: 2018-10-03 01:35:36.527251
Total Rows Read: 21000000, Paired Rows: 2191878, Time: 2018-10-03 01:35:49.235037
Cleanin up!
Total Rows Read: 21100000, Paired Rows: 2202050, Time: 2018-10-03 01:36:48.015164
Total Rows Read: 21200000, Paired Rows: 2212710, Time: 2018-10-03 01:37:00.609242
Total Rows Read: 21300000, Paired Rows: 2223647, Time: 2018-10-03 01:37:13.275885
Tota

Total Rows Read: 30100000, Paired Rows: 3149651, Time: 2018-10-03 08:24:25.526103
Total Rows Read: 30200000, Paired Rows: 3159418, Time: 2018-10-03 08:24:38.350983
Total Rows Read: 30300000, Paired Rows: 3169497, Time: 2018-10-03 08:24:51.048722
Total Rows Read: 30400000, Paired Rows: 3179259, Time: 2018-10-03 08:25:03.716539
Total Rows Read: 30500000, Paired Rows: 3189093, Time: 2018-10-03 08:25:16.495522
Total Rows Read: 30600000, Paired Rows: 3198690, Time: 2018-10-03 08:25:29.203977
Total Rows Read: 30700000, Paired Rows: 3208551, Time: 2018-10-03 08:25:42.104449
Total Rows Read: 30800000, Paired Rows: 3218495, Time: 2018-10-03 08:25:55.042517
Total Rows Read: 30900000, Paired Rows: 3228398, Time: 2018-10-03 08:26:08.328484
Total Rows Read: 31000000, Paired Rows: 3238296, Time: 2018-10-03 08:26:21.453460
Cleanin up!
Total Rows Read: 31100000, Paired Rows: 3248590, Time: 2018-10-03 08:27:57.199050
Total Rows Read: 31200000, Paired Rows: 3258832, Time: 2018-10-03 08:28:10.834412
Tota

Total Rows Read: 40000000, Paired Rows: 4192083, Time: 2018-10-03 09:36:05.478140
Cleanin up!
Total Rows Read: 40100000, Paired Rows: 4202679, Time: 2018-10-03 09:38:11.424730
Total Rows Read: 40200000, Paired Rows: 4213450, Time: 2018-10-03 09:38:24.037523
Total Rows Read: 40300000, Paired Rows: 4223904, Time: 2018-10-03 09:38:36.713537
Total Rows Read: 40400000, Paired Rows: 4234009, Time: 2018-10-03 09:38:49.494459
Total Rows Read: 40500000, Paired Rows: 4244162, Time: 2018-10-03 09:39:02.288008
Total Rows Read: 40600000, Paired Rows: 4254494, Time: 2018-10-03 09:39:14.905153
Total Rows Read: 40700000, Paired Rows: 4264538, Time: 2018-10-03 09:39:27.256415
Total Rows Read: 40800000, Paired Rows: 4274654, Time: 2018-10-03 09:39:39.923035
Total Rows Read: 40900000, Paired Rows: 4284544, Time: 2018-10-03 09:39:52.601010
Total Rows Read: 41000000, Paired Rows: 4294663, Time: 2018-10-03 09:40:05.344612
Cleanin up!
Total Rows Read: 41100000, Paired Rows: 4305026, Time: 2018-10-03 09:42:19

KeyboardInterrupt: 